# 転移学習

本章では、前回取り扱った犬猫の分類問題に対して、さらに精度を向上する方法として転移学習について解説します。

コンピュータビジョン（画像認識）に特化したライブラリである ChainerCV の使用方法も確認します。  


## 転移学習

ディープラーニングの学習は、通常、非常に多くの学習データを必要とします。
そのようなデータを使って学習するのも、非常に時間がかかります。
この問題を解決するためによく行われる方法のひとつとして転移学習（transfer learning）があります。
これは、モデルを一から構築して学習させるのではなく、既存の学習済みモデル（ソースモデル）をベースにして学習させるというものです。
これによって少ないデータでの学習が可能になり、短い学習時間で高い精度を得ることができます。

画像認識の領域では、[ImageNet](http://www.image-net.org/) と呼ばれる 1000 クラス一般物体認識のタスクで優秀な成績を収めたモデルが公開されており、これらをソースモデルとして使うことが一般的です。

なお、「転移学習」という言葉は、本章で扱うものよりももう少し広い意味を持ちます。本章では画像認識タスクで学習したものを画像認識タスクで再学習していますが、例えば、音声認識で学習したモデルを動画認識に使うといったようにドメインをまたぐ学習も転移学習とよびます。

## ネットワークアーキテクチャ

画像認識の代表的なモデルには下記が挙げられます。

- VGG16（2014）
- Google Net（2014）
- ResNet（2015）

今回は、比較的構造のシンプルな VGG16 をソースモデルとして使用した実装方法についてお伝えします。  

その他のモデルについては下記の公式ドキュメントを確認してください。  

- [GoogLeNet](https://docs.chainer.org/en/stable/reference/generated/chainer.links.GoogLeNet.html)
- [ResNet152](https://docs.chainer.org/en/stable/reference/generated/chainer.links.ResNet152Layers.html)
- [VGG16](https://docs.chainer.org/en/stable/reference/generated/chainer.links.VGG16Layers.html)
- [学習済みモデル一覧](https://docs.chainer.org/en/stable/reference/links.html#pre-trained-models)

下の図は、今回学習するネットワークの模式図です。

![ネットワークの構造](images/13/02.png)

図のようにソースモデルの途中の出力を取り出し、そこから新しく計算をつなげ、新しいモデル（ターゲットモデル）を組みます。

ソースモデルから、損失関数を計算する 1 つ前の層の出力を「特徴量」として取り出すことが最も一般的です。
このように得られた特徴量は、もとの入力変数よりも学習しやすい表現になっており、新しいモデルの学習が効率的になります。

今回は、そこに新しく全結合層を追加し、目的の 2 クラス（犬・猫）分類タスクに即した識別を行います。

## データの読み込み

### 必要なライブラリのインポート

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### データのダウンロードとディレクトリ構成の確認

今回使うデータをダウンロードし解凍します。

In [ ]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
!unzip -q -o kagglecatsanddogs_3367a.zip

解凍ができたら中身の確認をします。

In [ ]:
!ls -al ./PetImages/

`PetImages` というディレクトリの中に、`Cat` と `Dog` というディレクトリがあります。
それぞれのディレクトリの中がどのようになっているのか確認します。

In [ ]:
# Catの中のファイル5行だけを表示
!ls -l ./PetImages/Cat/ | head -5

# Dogの中のファイル5行だけを表示
!ls -l ./PetImages/Dog/ | head -5

`Cat` と `Dog` どちらのディレクトリもJPEGファイルが入っていることが確認できました。

### 画像データの読み込みと前処理

画像データを読み込み、Chainer および VGG16 モデルに適した形に変換します。  
画像の変換は、`chainer.links.model.vision.vgg.prepare()` という関数を使用すると簡単に行うことができます。  
この関数は単に画像を Chainer の形に合わせるだけではなく、いくつかの特殊な前処理を実施します。  

**VGG16 の前処理**

- (高さ、幅、チャンネル) の順になっている shape を (チャンネル、高さ、幅) の順に並び替える
- カラーチャンネルの順番を RGB から BGR に変換する
- 各画素の値から平均値を引く（中心化）
- 画像サイズを 224×224 に変換する

VGG16 に限らず、ネットワークによって最適な前処理を行う場合がほとんどです。  
前処理についてはデータオーグメンテーションの章で詳細をお伝えします。  

In [ ]:
from glob import glob

In [ ]:
# ファイルパスの取得
dog_filepaths = glob('./PetImages/Dog/*')
cat_filepaths = glob('./PetImages/Cat/*')
dog_filepaths = dog_filepaths[:1000]
cat_filepaths = cat_filepaths[:1000]

In [ ]:
import chainer
import chainer.links as L
import chainer.functions as F

In [ ]:
import cv2

`L.model.vision.vgg.prepare()` を使用する際の注意点として、入力する画像は下記の要件を満たす必要があります。  

- shape が (高さ、幅、チャンネル) の順になっていること
- カラーチャンネルの順番が RGB になっていること

今回 shape は問題ありませんが、チャンネルの順番が BGR になっているため RGB に変換しておきます。  

In [ ]:
# 画像の読み込みと BGR → RGB の変換
img = cv2.cvtColor(cv2.imread(dog_filepaths[0]), cv2.COLOR_BGR2RGB)

In [ ]:
# VGG16 の前処理の適応
img = L.model.vision.vgg.prepare(img)

画像がどのように変換されているのか確認します。

ChainerCV には NumPy 形式の画像データを簡単に表示する機能があります。ChainerCV は Google Colaboratoryで デフォルトで準備されていないため、 `pip` を使用してインストールを実行します。

In [ ]:
!pip install chainercv

In [ ]:
import chainercv

画像の表示には `chainercv.visualizations.vis_image()` という関数を使用します。

In [ ]:
# 前処理済みの画像を表示する
chainercv.visualizations.vis_image(img)

画像全体から平均値が引かれているため、目や口のように暗い部分は反転してしまっていますが、前処理が施されていることが確認できます。

In [ ]:
type(img)

In [ ]:
img.dtype

In [ ]:
img.shape

### データセットの作成

` L.model.vision.vgg.prepare()` の挙動を確認したところで、全ての画像に対して、処理を加えるプログラムを作成します。  

In [ ]:
imgs = []
labels = []
for dog_filepath in dog_filepaths:
  img = cv2.imread(dog_filepath)
  if img is None:
    continue
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = L.model.vision.vgg.prepare(img)
  imgs.append(img)
  labels.append(0)

In [ ]:
for cat_filepath in cat_filepaths:
  img = cv2.imread(cat_filepath)
  if img is None:
    continue
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = L.model.vision.vgg.prepare(img)
  imgs.append(img)
  labels.append(1)

In [ ]:
import numpy as np

In [ ]:
x = np.array(imgs)
t = np.array(labels, np.int32)

In [ ]:
# データセットの作成
from chainer.datasets import TupleDataset
dataset = TupleDataset(x, t)

In [ ]:
# 訓練と検証データへ分割
from chainer.datasets import split_dataset_random
n_train = int(len(dataset)*0.7)
train, test = split_dataset_random(dataset, n_train, seed=1)

In [ ]:
len(train)

In [ ]:
len(test)

### ネットワークの定義

繰り返しになりますが、今回学習するネットワークは次のようになります。

![ネットワークの構造](images/13/02.png)

Chainer には、ソースモデルである VGG16 は `L.VGG16Layers` として定義されています。

In [ ]:
vgg16 = L.VGG16Layers()

`available_layers` 属性で VGG16 の中の構造を確認することができます。

In [ ]:
# 構造の確認
vgg16.available_layers

VGG16 は、1000 クラスの一般物体認識タスクで学習されたモデルです。
`prob` は 1000 クラスの確率分布、その手前の `fc8` は 1000 クラスの確率化前の表現を出力します。
この 2 つの層は、1000 クラスの一般物体認識に特化した層ということです。

今回は、そのひとつ前の層、`fc7` の出力を特徴量として利用します。

モデルのクラスは次のように記述します。

In [ ]:
class VGG16(chainer.Chain):

    def __init__(self, n_out=9):
        super().__init__()
        with self.init_scope():
            self.base = L.VGG16Layers()
            self.fc = L.Linear(None, n_out)

    def forward(self, x):
        h = self.base(x, layers=['fc7'])
        h = self.fc(h['fc7'])
        return h

`__init__()` メソッドで、ソースモデルである `L.VGG16Layers` をインスタンス化し、`self.base` として定義します。

そして、新しい部分である全結合層 `fc` も定義します。  

順伝播の記述（`forward()`）は、ソースモデルの `fc7` 層までの計算を `self.base(x, layers=['fc7'])` と記述します。
この結果は `dict` になっており、`h['fc7']` のように、層の名前をキーにしてその層の出力を取り出すことができます。
そして、その出力を全結合層 `fc` に流すという計算の流れになっています。

In [ ]:
# 乱数のシード固定用の関数
import random

def reset_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    if chainer.cuda.available:
        chainer.cuda.cupy.random.seed(seed)

In [ ]:
# CPUとGPU関連のシードをすべて固定
reset_seed(0)

In [ ]:
# インスタンス化
vgg16 = VGG16()
model = L.Classifier(vgg16)

In [ ]:
gpu_id = 0 
model.to_gpu(gpu_id)

In [ ]:
# Optimizer の定義と model との紐づけ
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

ここまでは今までの実装と同じです。

## パラメータを固定する

定義したモデルをこれまで通り学習すると、ソースモデル部分のパラメータも学習が行われ、更新されます[<sup>*1</sup>]。
しかし、少ない学習データですべてのパラメータを学習すると、計算時間がかかるうえ、過学習によって汎化性能が低下してしまいます。
そこで、ソースモデルの入力側のパラメータを固定し、その一部だけを再学習することができます。

今回は、ソースモデル部分は、最後の層（`fc7`）のみを学習します。

デフォルトでは、すべてのパラメータを学習するような設定になっています。

下記の 1 行によって、一度ソースモデル部分を学習しないように設定します。

In [ ]:
model.predictor.base.disable_update()

さらに、`fc7` 層だけ学習するように設定し直します。

In [ ]:
model.predictor.base.fc7.enable_update()

これで、ソースモデル部分のうち `fc7` 層だけを再学習し、それ以外はパラメータを固定して使う、という設定になりました。

### 学習率を調整する
学習の内容にもよりますが、ファインチューニングでは基本的に学習済みのパラメータの更新は小さく、それ以外の部分の学習は大きくすることが一般的です
（学習済みモデルはある程度学習が進んでいるため、微調整を加えるだけで問題ないためです）。

そのように二つの異なる部分のパラメータの更新を変更するにはオプティマイザの更新の規則の部分の設定を変更します。

[`Adam`](https://docs.chainer.org/en/stable/reference/generated/chainer.optimizers.Adam.html) オプティマイザでは、`alpha` と呼ばれるハイパーパラメータが学習率に相当します。
下記では、 `alpha` を `1e-4` と小さめに設定しておき、 `fc` の追加した層の部分のみ `update_rule` を用いて、その 10 倍に設定することで調整を行っています。  

In [ ]:
# Optimizerの定義とmodelとの紐づけ
alpha = 1e-4

optimizer = chainer.optimizers.Adam(alpha=alpha)
optimizer.setup(model)

model.predictor['fc'].W.update_rule.hyperparam.alpha = alpha * 10
model.predictor['fc'].b.update_rule.hyperparam.alpha = alpha * 10

## 学習

これでモデルとオプティマイザの準備ができました。
学習してみましょう

In [ ]:
batchsize = 64
train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)

In [ ]:
from chainer import training
from chainer.training import extensions

epoch = 20

updater = training.StandardUpdater(train_iter, optimizer, device=gpu_id)

trainer = training.Trainer(updater, (epoch, 'epoch'), out='drive/My Drive/Colab Notebooks/result')

# バリデーション用のデータで評価
trainer.extend(extensions.Evaluator(test_iter, model, device=gpu_id))

# 学習結果の途中を表示する
trainer.extend(extensions.LogReport(trigger=(1, 'epoch'), log_name='dog-cat_transferlearning-vgg16'))

# １ エポックごとに結果をログファイルに出力する
trainer.extend(extensions.PrintReport(['epoch', 'iteration', 'main/accuracy', 'validation/main/accuracy', 'main/loss', 'validation/main/loss', 'elapsed_time']), trigger=(1, 'epoch'))

In [ ]:
trainer.run()

In [ ]:
import pandas as pd
import json

In [ ]:
with open('drive/My Drive/Colab Notebooks/result/dog-cat_transferlearning-vgg16') as f:
    result = pd.DataFrame(json.load(f))

In [ ]:
result.tail(10)

In [ ]:
# 精度 (accuracy)
result[['main/accuracy', 'validation/main/accuracy']].plot()

転移学習を行うことによって高い精度を得ることができました。

今回はソースモデルの最後の層だけ再学習しましたが、もっと上の層のパラメータまで再学習することも可能です。しかし、学習するパラメータを増やしすぎると過学習しやすくなり、検証用データセットでの認識率が低下します。
余力のある読者は試してみるとよいでしょう。

## 脚注
<span id="fn1"><sup>*1</sup>: <small>このように既存のモデルを再学習することをファインチューニングといいます。</small></span>